1\. Készítsünk sorszámozott listát a szálláshelyek adatairól! 

a. A lista legyen a szálláshely típusa szerint, azon belül hely szerint, majd a neve szerint növekvően rendezve! 

b. A számozás típusonként kezdődjön újra!

In [ ]:
select *, row_number() 
over(partition by tipus order by tipus,hely, szallas_nev)
from szallashely

3\. Készítsük listát, amely megjeleníti az ügyfelek azonosítóját, nevét és a foglalásainak kezdő- és befejező dátumát, valamint azt, hogy a foglalás összesen hány főre történt! 

a. Egy új oszlopban jelenítsük meg azt is, hogy az ügyfél előző három foglalása összesen hány főre történt!

In [ ]:
select u.[login], u.nev, f.mettol, f.meddig, f.felnott_szam, f.gyermek_szam, 
       sum(f.felnott_szam + f.gyermek_szam) over(partition by u.[login] order by f.mettol rows between 3 preceding and 1 preceding) 
from foglalas f 
inner join ugyfel u on f.ugyfel_fk = u.[login]

<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">6. Alakítsa át a melléklet 1. pontjában lévő lekérdezést úgy, hogy JOIN helyett CROSS APPLY-t alkalmaz!</span>  
<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">a. Válaszként adja meg az átalakított lekérdezés kódját!</span>

<span style="color: #0000ff;">SELECT</span> rt.TERMEKKOD, 

       <span style="color: #795e26;">AVG</span>(rt.EGYSEGAR) <span style="color: #0000ff;">AS</span> <span style="color: #a31515;">'Atlagos&nbsp;ar'</span>

<span style="color: #0000ff;">FROM</span> Rendeles\_tetel rt <span style="color: #0000ff;">JOIN</span> Rendeles r

     <span style="color: #0000ff;">ON</span> rt.SORSZAM <span style="color: #000000;">=</span> r.SORSZAM

<span style="color: #0000ff;">GROUP&nbsp;BY</span> rt.TERMEKKOD

In [ ]:
SELECT rt.TERMEKKOD, 

       AVG(rt.EGYSEGAR) AS 'Atlagos ar'

FROM Rendeles_tetel rt CROSS APPLY
(
    select * from rendeles r
    WHERE rt.SORSZAM = r.SORSZAM
) r
GROUP BY rt.TERMEKKOD

<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">7. Alakítsa át a melléklet 2. pontjában lévő lekérdezést úgy, hogy mindkét (INNER) JOIN részbe sorra beilleszti a tanult JOIN HINT-eket (először mindkét helyre MERGE, majd HASH, utána LOOP, végül REMOTE).</span>  
<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">a. Melyik hint használata adja a legjobb megoldást?</span>  
<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">b. Mi lesz ezt a hint-et tartalmazó lekérdezés kódja?</span>

```
SELECT rt.*,
       u.nev,
       SUM(rt.mennyiseg*rt.egysegar) 
	   OVER(PARTITION BY r.LOGIN ORDER BY rt.SORSZAM, termekkod 
	     RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
	   AS 'Eddigi rendel�si t�telek �ssz�rt�ke'
FROM Rendeles_tetel rt JOIN Rendeles r 
     ON rt.SORSZAM = r.SORSZAM
	 JOIN Ugyfel u ON r.LOGIN = u.LOGIN
```

In [ ]:
SELECT rt.*,
       u.nev,
       SUM(rt.mennyiseg*rt.egysegar) 
	   OVER(PARTITION BY r.LOGIN ORDER BY rt.SORSZAM, termekkod 
	     RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
	   AS 'Eddigi rendel�si t�telek �ssz�rt�ke'
FROM Rendeles_tetel rt INNER HASH JOIN Rendeles r 
     ON rt.SORSZAM = r.SORSZAM
	 JOIN Ugyfel u ON r.LOGIN = u.LOGIN

<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">8. Készítsen non-clustered indexet a Rendeles_tetel táblához!</span>  
<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">a. Az index neve NCI_sorszam_termekkod legyen</span>  
<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">b. A sorszam legyen az index kulcs oszlop, a termékkód pedig az indexbe belefoglalt oszlop</span>  
<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">c. Válaszként adja meg az indexet generáló script-et!</span>